# TORONTO NEIGHBORHOODS

## Part One: Toronto Neighborhood Dataframe

In [1]:
import pandas as pd
import numpy as np

In [2]:
# get the table from Wikipedia
toronto_neigh, = pd.read_html("http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header = 0, match = 'Neighborhood')
toronto_neigh.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [3]:
# change column name
toronto_neigh.rename(columns = {'Postal code': 'PostalCode'}, inplace = True)
toronto_neigh.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [4]:
# drop the rows without assigned borough
toronto_neigh.drop(toronto_neigh[toronto_neigh['Borough'] == 'Not assigned'].index, inplace = True)
toronto_neigh.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [5]:
# check if there are multiple rows with the same postal code
if toronto_neigh['PostalCode'].is_unique:
    print('No action needed.')
else:
    print('Need to group by the postal code.')

No action needed.


In [6]:
# raplace NaN in Neighbohood with Borough
toronto_neigh['Neighborhood'].replace(np.nan, toronto_neigh['Borough'],inplace = True)
toronto_neigh.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [7]:
# replace / with comma in Neighborhood
toronto_neigh['Neighborhood'] = toronto_neigh['Neighborhood'].str.replace('/', ',')
toronto_neigh.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [8]:
# size of the dataframe
print(toronto_neigh.shape)

(103, 3)


## Part Two: Latitude and Longitude of the Neighborhoods

In [9]:
# import the geospatial coordinates
lat_long = pd.read_csv('http://cocl.us/Geospatial_data', header = 0)
lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
# sort the postal code in Toronto neighborhoods. 
toronto_neigh.sort_values(by = 'PostalCode', axis = 0, ascending = True, inplace = True)
toronto_neigh.reset_index(drop = True, inplace = True)

In [11]:
# check if they are in the same order as those in lat_long. If true, add the coordinates to toronto_neigh dataframe.
if all(toronto_neigh['PostalCode'] == lat_long['Postal Code']):
    print('The Orders of the postal codes are the same. Concatenate the coordinates dataframe.')
    toronto_neigh_geo = pd.concat([toronto_neigh, lat_long], axis = 1 )
else:
    print('Something wrong with the orders.')

toronto_neigh_geo.drop(columns= 'Postal Code', inplace = True)

The Orders of the postal codes are the same. Concatenate the coordinates dataframe.


In [12]:
toronto_neigh_geo.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [13]:
print(toronto_neigh_geo.shape)

(103, 5)


## Part Three: Neighborhood Clustering

#### 1. Map of Toronto

In [14]:
# work with only boroughs that contain the word Toronto
borough_toronto = toronto_neigh_geo[toronto_neigh_geo['Borough'].str.contains('Toronto', case = False)].reset_index(drop = True)
borough_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [15]:
!pip install folium
import folium
from geopy.geocoders import Nominatim

In [16]:
# see Toronto neighborhoods on a map
address = 'Toronto, ON'
geolocator = Nominatim(user_agent = "on_explorer")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude

# create map object
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=12)

# add markers to map
for lat, lng, label in zip(borough_toronto['Latitude'], borough_toronto['Longitude'], borough_toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)  
    
map_toronto

#### 2. Find the venues around the neighborhoods from Foursquare

In [17]:
import requests

In [18]:
# The code was removed by Watson Studio for sharing.

In [19]:
# this function was created by Alex Aklson and Polong Lin with slight modifications
def getNearbyVenues(names, latitudes, longitudes, LIMIT = 100, radius = 500):
    
    '''
    
    names: neighborhood names,
    latitudes, longitudes: geospatial coordinates of the neighborhoods,
    Limit: maximum number of venues returned,
    radius: search radius in meters.
    
    '''
    
    venues_list = []
    count = 0
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        count += 1

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    #show number of neighborhood explored
    print('\n'+ " ".join([str(count), 'neighborhoods explored.']))
    
    return(nearby_venues)

In [20]:
toronto_venues = getNearbyVenues(names = borough_toronto['Neighborhood'],
                                   latitudes = borough_toronto['Latitude'],
                                   longitudes = borough_toronto['Longitude']
                                  )
print(toronto_venues.shape)
toronto_venues.head()

The Beaches
The Danforth West , Riverdale
India Bazaar , The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park , Summerhill East
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
Rosedale
St. James Town , Cabbagetown
Church and Wellesley
Regent Park , Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond , Adelaide , King
Harbourfront East , Union Station , Toronto Islands
Toronto Dominion Centre , Design Exchange
Commerce Court , Victoria Hotel
Roselawn
Forest Hill North & West
The Annex , North Midtown , Yorkville
University of Toronto , Harbord
Kensington Market , Chinatown , Grange Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Stn A PO Boxes
First Canadian Place , Underground city
Christie
Dufferin , Dovercourt Village
Little Portugal , Trinity
Brockton , Parkdale Village , Exhibition Place
High Park , 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West , Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


#### 3. Analyzing the venues in the neighborhoods

In [21]:
# check numbers of venues returned for each neighborhood
toronto_venues_grouped = toronto_venues.groupby('Neighborhood', sort = False)[['Venue']].count()
toronto_venues_grouped

,Venue
Neighborhood,
The Beaches,4
"The Danforth West , Riverdale",43
"India Bazaar , The Beaches West",22
Studio District,40
Lawrence Park,3
Davisville North,8
North Toronto West,18
Davisville,37
"Moore Park , Summerhill East",2


##### If we look at the dataframe above, there are neighborhoods that have only few venues. If we only consider neighborhoods with more than ten venues as attractive, let's drop the unattractive neighborhoods.

In [22]:
# get the list of neighborhoods with the number of venues less than 11
neigh_drop_list = toronto_venues_grouped.index[toronto_venues_grouped['Venue'] <= 10].tolist()
neigh_drop_list

['The Beaches',
 'Lawrence Park',
 'Davisville North',
 'Moore Park , Summerhill East',
 'Rosedale',
 'Roselawn',
 'Forest Hill North & West']

In [23]:
# drop the neighborhoods
toronto_venues_mod = toronto_venues.set_index('Neighborhood').drop(labels = neigh_drop_list, axis = 0).reset_index()
print('There are {} neighborhoods remained'.format(len(toronto_venues_mod['Neighborhood'].unique())))
toronto_venues_mod.head()

There are 32 neighborhoods remained


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"The Danforth West , Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
1,"The Danforth West , Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
2,"The Danforth West , Riverdale",43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant
3,"The Danforth West , Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
4,"The Danforth West , Riverdale",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop


In [24]:
# get the number of unique venue categories
print('There are {} unique venue categories.'.format(len(toronto_venues_mod['Venue Category'].unique())))

There are 227 unique venue categories.


In [25]:
# one hot encoding for the venues in Toronto
toronto_onehot = pd.get_dummies(toronto_venues_mod[['Venue Category']], columns = ['Venue Category'], prefix = "", prefix_sep = "")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood_name'] = toronto_venues_mod['Neighborhood'] 

# move neighborhood column to the first column
col = toronto_onehot.columns.tolist()
toronto_onehot = toronto_onehot[col[-1:] + col[:-1]]

toronto_onehot.head()

,Neighborhood_name,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"The Danforth West , Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"The Danforth West , Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"The Danforth West , Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"The Danforth West , Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West , Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### let's find out the occurance of each venue category

In [26]:
toronto_grouped = toronto_onehot.groupby('Neighborhood_name').mean().reset_index()
toronto_grouped

,Neighborhood_name,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton , Parkdale Village , Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Business reply mail Processing CentrE,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824
3,"CN Tower , King and Spadina , Railway Lands , ...",0.066667,0.066667,0.066667,0.133333,0.2,0.066667,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.015385,0.000000,0.000000,0.000000,0.000000,0.015385
5,Christie,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.013889,0.000000,0.00,...,0.013889,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778
7,"Commerce Court , Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.054054,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Dufferin , Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


##### get the top 15 most common venues in each neighborhood

In [27]:
# define a function that return list of venue categories in descending order of popularity
def return_most_common_venues(row, num_top_venues):
    '''
    
    row: neighborhood,
    num_top_venues: number of venue categories returned.
    
    '''
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
# return a dataframe showing the top 10 most common venues categories
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns = columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood_name']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Berczy Park,Coffee Shop,Cheese Shop,Restaurant,Bakery,Beer Bar,Seafood Restaurant,Farmers Market,Cocktail Bar,Italian Restaurant,Café,Park,Clothing Store,Breakfast Spot,Eastern European Restaurant,Sporting Goods Shop
1,"Brockton , Parkdale Village , Exhibition Place",Café,Coffee Shop,Breakfast Spot,Gym,Burrito Place,Nightclub,Restaurant,Italian Restaurant,Intersection,Stadium,Bar,Climbing Gym,Bakery,Office,Convenience Store
2,Business reply mail Processing CentrE,Yoga Studio,Pizza Place,Skate Park,Smoke Shop,Farmers Market,Fast Food Restaurant,Spa,Burrito Place,Restaurant,Light Rail Station,Auto Workshop,Brewery,Garden Center,Comic Shop,Garden
3,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Airport,Harbor / Marina,Plane,Sculpture Garden,Boat or Ferry,Bar,Boutique,Airport Terminal,Airport Food Court,Airport Gate,Gift Shop,Doner Restaurant,Fast Food Restaurant
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Salad Place,Restaurant,Ice Cream Shop,Bubble Tea Shop,Fried Chicken Joint,Japanese Restaurant,Burger Joint,Middle Eastern Restaurant,French Restaurant,Modern European Restaurant,Ramen Restaurant
5,Christie,Grocery Store,Café,Park,Coffee Shop,Baby Store,Athletics & Sports,Diner,Nightclub,Restaurant,Candy Store,Gas Station,Italian Restaurant,Doner Restaurant,Donut Shop,Distribution Center
6,Church and Wellesley,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Restaurant,Yoga Studio,Dance Studio,Burger Joint,Pub,Mediterranean Restaurant,Hotel,Men's Store,Gay Bar,Gastropub,Thai Restaurant,Theme Restaurant
7,"Commerce Court , Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Deli / Bodega,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Gastropub,Bar,Beer Bar,Thai Restaurant,Vegetarian / Vegan Restaurant
8,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Café,Coffee Shop,Sushi Restaurant,Gym,Italian Restaurant,Toy / Game Store,Farmers Market,Indoor Play Area,Seafood Restaurant,Indian Restaurant,Spa,Diner
9,"Dufferin , Dovercourt Village",Bakery,Pharmacy,Middle Eastern Restaurant,Music Venue,Café,Bar,Bank,Supermarket,Brewery,Grocery Store,Gym / Fitness Center,Park,Distribution Center,Event Space,Ethiopian Restaurant


#### 4. Clustering using k-means

In [29]:
from sklearn.cluster import KMeans 

In [30]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood_name', axis = 1)

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, init = 'k-means++', n_init = 12, random_state = 0).fit(toronto_grouped_clustering)

# add clustering labels to neighborhoods_venues_sorted
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#manhattan_merged.head() # check the last columns!


In [31]:
# drop the neighborhood with number of venues less than 11
toronto_clustered = borough_toronto.set_index('Neighborhood').drop(labels = neigh_drop_list, axis = 0).sort_values(by = 'Neighborhood', axis = 0, ascending = True).reset_index()

#merge with neighborhoods_venues_sorted
toronto_clustered = toronto_clustered.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_clustered

,Neighborhood,PostalCode,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Berczy Park,M5E,Downtown Toronto,43.644771,-79.373306,4,Coffee Shop,Cheese Shop,Restaurant,Bakery,...,Seafood Restaurant,Farmers Market,Cocktail Bar,Italian Restaurant,Café,Park,Clothing Store,Breakfast Spot,Eastern European Restaurant,Sporting Goods Shop
1,"Brockton , Parkdale Village , Exhibition Place",M6K,West Toronto,43.636847,-79.428191,0,Café,Coffee Shop,Breakfast Spot,Gym,...,Nightclub,Restaurant,Italian Restaurant,Intersection,Stadium,Bar,Climbing Gym,Bakery,Office,Convenience Store
2,Business reply mail Processing CentrE,M7Y,East Toronto,43.662744,-79.321558,4,Yoga Studio,Pizza Place,Skate Park,Smoke Shop,...,Fast Food Restaurant,Spa,Burrito Place,Restaurant,Light Rail Station,Auto Workshop,Brewery,Garden Center,Comic Shop,Garden
3,"CN Tower , King and Spadina , Railway Lands , ...",M5V,Downtown Toronto,43.628947,-79.394420,2,Airport Service,Airport Lounge,Airport,Harbor / Marina,...,Sculpture Garden,Boat or Ferry,Bar,Boutique,Airport Terminal,Airport Food Court,Airport Gate,Gift Shop,Doner Restaurant,Fast Food Restaurant
4,Central Bay Street,M5G,Downtown Toronto,43.657952,-79.387383,4,Coffee Shop,Italian Restaurant,Sandwich Place,Café,...,Restaurant,Ice Cream Shop,Bubble Tea Shop,Fried Chicken Joint,Japanese Restaurant,Burger Joint,Middle Eastern Restaurant,French Restaurant,Modern European Restaurant,Ramen Restaurant
5,Christie,M6G,Downtown Toronto,43.669542,-79.422564,0,Grocery Store,Café,Park,Coffee Shop,...,Athletics & Sports,Diner,Nightclub,Restaurant,Candy Store,Gas Station,Italian Restaurant,Doner Restaurant,Donut Shop,Distribution Center
6,Church and Wellesley,M4Y,Downtown Toronto,43.665860,-79.383160,4,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Restaurant,...,Dance Studio,Burger Joint,Pub,Mediterranean Restaurant,Hotel,Men's Store,Gay Bar,Gastropub,Thai Restaurant,Theme Restaurant
7,"Commerce Court , Victoria Hotel",M5L,Downtown Toronto,43.648198,-79.379817,4,Coffee Shop,Restaurant,Café,Hotel,...,Gym,Deli / Bodega,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Gastropub,Bar,Beer Bar,Thai Restaurant,Vegetarian / Vegan Restaurant
8,Davisville,M4S,Central Toronto,43.704324,-79.388790,1,Pizza Place,Dessert Shop,Sandwich Place,Café,...,Sushi Restaurant,Gym,Italian Restaurant,Toy / Game Store,Farmers Market,Indoor Play Area,Seafood Restaurant,Indian Restaurant,Spa,Diner
9,"Dufferin , Dovercourt Village",M6H,West Toronto,43.669005,-79.442259,0,Bakery,Pharmacy,Middle Eastern Restaurant,Music Venue,...,Bar,Bank,Supermarket,Brewery,Grocery Store,Gym / Fitness Center,Park,Distribution Center,Event Space,Ethiopian Restaurant


##### visualize the cluster

In [32]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [33]:
# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
for lat, lon, neigh, cluster in zip(toronto_clustered['Latitude'], toronto_clustered['Longitude'], toronto_clustered['Neighborhood'], toronto_clustered['Cluster Labels']):
    label = folium.Popup(str(neigh) + '\n\n' 'Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### 5. Examine the clusters

##### cluster 1

In [34]:
toronto_clustered.loc[toronto_clustered['Cluster Labels'] == 0, toronto_clustered.columns[[0] + list(range(6, toronto_clustered.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
1,"Brockton , Parkdale Village , Exhibition Place",Café,Coffee Shop,Breakfast Spot,Gym,Burrito Place,Nightclub,Restaurant,Italian Restaurant,Intersection,Stadium,Bar,Climbing Gym,Bakery,Office,Convenience Store
5,Christie,Grocery Store,Café,Park,Coffee Shop,Baby Store,Athletics & Sports,Diner,Nightclub,Restaurant,Candy Store,Gas Station,Italian Restaurant,Doner Restaurant,Donut Shop,Distribution Center
9,"Dufferin , Dovercourt Village",Bakery,Pharmacy,Middle Eastern Restaurant,Music Venue,Café,Bar,Bank,Supermarket,Brewery,Grocery Store,Gym / Fitness Center,Park,Distribution Center,Event Space,Ethiopian Restaurant
13,"High Park , The Junction South",Thai Restaurant,Mexican Restaurant,Café,Bakery,Bar,Diner,Fried Chicken Joint,Flea Market,Italian Restaurant,Furniture / Home Store,Bookstore,Arts & Crafts Store,Cajun / Creole Restaurant,Park,Antique Shop
26,Studio District,Café,Coffee Shop,Bakery,Gastropub,American Restaurant,Brewery,Yoga Studio,Cheese Shop,Fish Market,Bookstore,Italian Restaurant,Sandwich Place,Latin American Restaurant,Clothing Store,Diner
28,"The Annex , North Midtown , Yorkville",Café,Sandwich Place,Coffee Shop,Pub,BBQ Joint,Pizza Place,History Museum,Pharmacy,Indian Restaurant,Park,Middle Eastern Restaurant,American Restaurant,Liquor Store,Burger Joint,Vegetarian / Vegan Restaurant
31,"University of Toronto , Harbord",Café,Yoga Studio,Bakery,Restaurant,Japanese Restaurant,Bookstore,Bar,Italian Restaurant,Sandwich Place,Sushi Restaurant,Pub,Video Game Store,Chinese Restaurant,Beer Store,Beer Bar


These neighborhoods have many cafés and restaurants. They are busy neighborhoods, so there might be high level of noise.

##### cluster 2

In [35]:
toronto_clustered.loc[toronto_clustered['Cluster Labels'] == 1, toronto_clustered.columns[[0] + list(range(6, toronto_clustered.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
8,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Café,Coffee Shop,Sushi Restaurant,Gym,Italian Restaurant,Toy / Game Store,Farmers Market,Indoor Play Area,Seafood Restaurant,Indian Restaurant,Spa,Diner
14,"India Bazaar , The Beaches West",Park,Sandwich Place,Fast Food Restaurant,Pet Store,Brewery,Burrito Place,Restaurant,Pub,Pizza Place,Gym,Sushi Restaurant,Movie Theater,Liquor Store,Italian Restaurant,Fish & Chips Shop
22,"Runnymede , Swansea",Café,Coffee Shop,Pizza Place,Sushi Restaurant,Pub,Italian Restaurant,Tea Room,Yoga Studio,Bank,Bar,Food,Indie Movie Theater,Fish & Chips Shop,Dessert Shop,Bookstore


These neighborhoods have many fast food places such as pizza and dessert places.

##### cluster 3

In [36]:
toronto_clustered.loc[toronto_clustered['Cluster Labels'] == 2, toronto_clustered.columns[[0] + list(range(6, toronto_clustered.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
3,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Airport,Harbor / Marina,Plane,Sculpture Garden,Boat or Ferry,Bar,Boutique,Airport Terminal,Airport Food Court,Airport Gate,Gift Shop,Doner Restaurant,Fast Food Restaurant


This cluster shows the neighborhood is near the airport.

##### cluster 4

In [37]:
toronto_clustered.loc[toronto_clustered['Cluster Labels'] == 3, toronto_clustered.columns[[0] + list(range(6, toronto_clustered.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
18,"Parkdale , Roncesvalles",Gift Shop,Bookstore,Movie Theater,Eastern European Restaurant,Italian Restaurant,Bar,Dog Run,Restaurant,Dessert Shop,Breakfast Spot,Coffee Shop,Cuban Restaurant,Dance Studio,Donut Shop,Falafel Restaurant


This neighborhood has many grocery stores and food places. It is good to live in.

##### cluster 5

In [38]:
toronto_clustered.loc[toronto_clustered['Cluster Labels'] == 4, toronto_clustered.columns[[0] + list(range(6, toronto_clustered.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Berczy Park,Coffee Shop,Cheese Shop,Restaurant,Bakery,Beer Bar,Seafood Restaurant,Farmers Market,Cocktail Bar,Italian Restaurant,Café,Park,Clothing Store,Breakfast Spot,Eastern European Restaurant,Sporting Goods Shop
2,Business reply mail Processing CentrE,Yoga Studio,Pizza Place,Skate Park,Smoke Shop,Farmers Market,Fast Food Restaurant,Spa,Burrito Place,Restaurant,Light Rail Station,Auto Workshop,Brewery,Garden Center,Comic Shop,Garden
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Salad Place,Restaurant,Ice Cream Shop,Bubble Tea Shop,Fried Chicken Joint,Japanese Restaurant,Burger Joint,Middle Eastern Restaurant,French Restaurant,Modern European Restaurant,Ramen Restaurant
6,Church and Wellesley,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Restaurant,Yoga Studio,Dance Studio,Burger Joint,Pub,Mediterranean Restaurant,Hotel,Men's Store,Gay Bar,Gastropub,Thai Restaurant,Theme Restaurant
7,"Commerce Court , Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Deli / Bodega,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Gastropub,Bar,Beer Bar,Thai Restaurant,Vegetarian / Vegan Restaurant
10,"First Canadian Place , Underground city",Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Seafood Restaurant,Bar,Deli / Bodega,Asian Restaurant,Japanese Restaurant,Steakhouse,Thai Restaurant,Sushi Restaurant,Tea Room
11,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Middle Eastern Restaurant,Bubble Tea Shop,Japanese Restaurant,Restaurant,Cosmetics Shop,Bookstore,Fast Food Restaurant,Diner,Tea Room,Thai Restaurant,Theater,Italian Restaurant
12,"Harbourfront East , Union Station , Toronto Is...",Coffee Shop,Aquarium,Café,Restaurant,Italian Restaurant,Hotel,Scenic Lookout,Sporting Goods Shop,Fried Chicken Joint,Brewery,Bar,Pizza Place,Music Venue,Park,Baseball Stadium
15,"Kensington Market , Chinatown , Grange Park",Café,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Bar,Vegetarian / Vegan Restaurant,Gaming Cafe,Bakery,Grocery Store,Dessert Shop,Breakfast Spot,Filipino Restaurant,Belgian Restaurant,Farmers Market,Pharmacy
16,"Little Portugal , Trinity",Bar,Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Men's Store,Asian Restaurant,Café,Juice Bar,Ice Cream Shop,Bistro,Brewery,Italian Restaurant,Japanese Restaurant,Record Shop,Yoga Studio


These neighborhoods have many coffee shops, restaurants and parks. They are suitable for people who prefer slow living.